In [1]:
# torch 함수 불러오기
# onnx로 변환하기
# 변환된 onnx 검증하기

In [2]:
import numpy as np

# torch 함수 불러오기
import torch
from add_ai3 import Encoder
from add_ai3 import Decoder
from add_ai3 import Encoder_n_Decoder
# onnx로 변환하기
import torch.onnx
# 변환된 onnx 검증하기
import onnx # 검증만
import onnxruntime # 실제 계산까지

In [3]:
# 함수 불러오기
file_name = "data/add_ai3"

F = torch.load(file_name + '.pt', weights_only=False, map_location='cpu') # cpu로 불러오기
F.eval() # dropout 끄기

print(F)

Encoder_n_Decoder(
  (encoder): Encoder(
    (embedding): Embedding(12, 11, padding_idx=0)
    (rnn): LSTM(11, 11, batch_first=True, bidirectional=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(12, 11, padding_idx=0)
    (rnn): LSTM(11, 11, batch_first=True)
    (f): Linear(in_features=11, out_features=12, bias=True)
  )
)


In [4]:
# onnx로 변환하기
x = torch.randint(0, 12, (1,7)).type(torch.long) # 입력값 더미 텐서
dynamic_axes = {'x' : {0 : 'b', 1 : 'f'}, 'y' : {0 : 'b'}} # 가변길이 차원 지정 dict

torch.onnx.export(
    F,
    x,
    file_name + '.onnx',
    export_params=True,
    opset_version=20,
    do_constant_folding=True,
    input_names=['x'],
    output_names=['y'],
    dynamic_axes=dynamic_axes
)

C:\ProgramData\anaconda3\Lib\site-packages\torch\onnx\symbolic_opset9.py:4277: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(


In [5]:
# onnx 확인하기
onnx_F = onnx.load(file_name + '.onnx')

onnx.checker.check_model(onnx_F) #문제 없이 저장이 되었으면 오류 메세지 없음

In [6]:
# 실제 계산값 비교
onnx_F = onnxruntime.InferenceSession(file_name + '.onnx', providers=["CPUExecutionProvider"])
np_x = x.numpy()

y = F(x) # torch 계산 값

inputs = {onnx_F.get_inputs()[0].name : np_x}
outputs = onnx_F.run(None, inputs)
onnx_y = outputs[0] # onnx 계산 값

print(y.detach().numpy())
print(onnx_y)

np.testing.assert_allclose(y.detach().numpy(), onnx_y, rtol=1e-03, atol=1e-05) #계산 결과가 오차범위 내 차이 없으면 오류 메세지 없음

[[[ -4.653347    15.551586     4.016596    -8.378742   -11.084572
   -10.094458    -9.3693905   -5.136682     2.1106915   11.235995
    13.876137    -7.186219  ]
  [ -6.944753    10.291261    13.765766    14.775002    11.442588
    -0.81896114 -13.110521   -18.0915     -16.792225    -9.171224
     6.820837   -16.395786  ]
  [-10.988034     9.514807    18.785196    19.970959    14.786121
    -1.4053516  -18.219189   -26.284159   -22.684204   -10.169791
     4.5456896  -17.916883  ]
  [  2.4043918   13.664791    16.163265    13.507683     7.6822906
    -5.1210337  -20.54197    -28.317373   -19.846909    -4.5340366
     7.996483   -17.38576   ]]]
[[[ -4.6533403  15.551567    4.016574   -8.378764  -11.08458
   -10.094446   -9.369357   -5.136647    2.1107183  11.236009
    13.876127   -7.1862206]
  [ -6.944751   10.29125    13.765756   14.775003   11.442604
    -0.8189323 -13.110493  -18.09149   -16.792233   -9.171246
     6.820812  -16.39579  ]
  [-10.988022    9.514816   18.785234   19.97